## Pipeline steps:
1. Load data, prepare for fit
2. Load model and other objects for training
3. Fit model, get report
4. Load and process test data
5%load_ext autoreload. Make submission

In [1]:
!python3 -m deeppavlov install squad_bert

2021-04-13 22:20:44.67 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'squad_bert' as '/home/gldsn/.local/share/virtualenvs/nti-ai-pipeline-LA9ij60T/lib/python3.7/site-packages/deeppavlov/configs/squad/squad_bert.json'
  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-xhi_jyn3


In [2]:
import os
from torch import cuda, device

BASE_DIR = "./../../../"
DATA_DIR = os.path.join(BASE_DIR, "data/rucos")
SOURCES_DIR = os.path.join(BASE_DIR, "nti_ai_pipeline")
TRAIN_FILE_PATH = os.path.join(DATA_DIR, "rucos_train.jsonl")
VAL_FILE_PATH = os.path.join(DATA_DIR, "rucos_val.jsonl")
TEST_FILE_PATH = os.path.join(DATA_DIR, "rucos_test.jsonl")

BATCH_SIZE = 2
DEVICE = device("cuda" if cuda.is_available() else "cpu")
TRANSFORMER_NAME = "DeepPavlov/rubert-base-cased"

In [3]:
import sys
import importlib
import torch

sys.path.append(SOURCES_DIR)

import nti_rucos
import pipeline
from torch.utils import data as torch_data
from nti_rucos import modeling as rucos_modeling
from pipeline.saving import local_saver



my_modules = [nti_rucos, pipeline, rucos_modeling, local_saver]

for module in my_modules:
    importlib.reload(module)

In [4]:
container = nti_rucos.data.RucosContainer(TRAIN_FILE_PATH, VAL_FILE_PATH)
train_data, test_data = container.train_test_split(shuffle=True)

train_dataset = nti_rucos.data.RucosTrainDataset(train_data)
val_dataset = nti_rucos.data.RucosValDataset(test_data)

train_loader = torch_data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
val_loader = torch_data.DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [5]:
proc = nti_rucos.modeling.RucosProcessor(TRANSFORMER_NAME, use_ner=False)
model = rucos_modeling.SentPairBinaryClassifier(
    TRANSFORMER_NAME,
    droprate=0.2, 
    use_ner=False
)
saver = pipeline.saving.local_saver.LocalSaver(save_dir='./saved_models')

manager = pipeline.modeling.ModelManager(model, proc, DEVICE, saver)

In [6]:
validator = nti_rucos.evaluating.RucosValidator()
# eval_val = validator.eval(manager, val_loader)

In [7]:
weights_updater = pipeline.WeightsUpdater(lr=7e-6, warmup=200, lr_end=1.7e-6, accum_iters=6, weight_decay=0.04, optimizer_class=torch.optim.AdamW) # NOTE: had set to 0 from 1e-2
trainer = pipeline.training.Trainer(validator, weights_updater)

In [8]:
# del model, manager
import gc, torch
gc.collect()
torch.cuda.empty_cache()

trainer.fit(train_loader, val_loader, manager, max_step=50001, steps_betw_evals=5000, stop_patience=3)

batch [RucosSampleFeatures(text1=('На заседании в четверг, 16 ноября, комитет по соответствию МОК признал Российское антидопинговое агентство "не соответствующим кодексу" организации.', 'На заседании в четверг, 16 ноября, комитет по соответствию Лозанне признал Российское антидопинговое агентство "не соответствующим кодексу" организации.'), text2=('На заседании МОК в Лозанне 5 декабря будет решено, какие санкции ждут РФ за манипуляции с допинг-пробами. Среди возможных вариантов - отстранение от участия в Олимпиаде в Пхенчхане. Международный олимпийский комитет (МОК) примет решение по поводу санкций в отношении России за мошенничество с допинг-пробами на Олимпиаде в Сочи в 2014 году на заседании в Лозанне, которое состоится 5 декабря. Об этом МОК сообщил в пятницу, 17 ноября. Всемирное антидопинговое агентство в преддверии Олимпиады в Пхенчхане решило не реаблитировать Российское антидопинговое агентство, сообщили в WADA по итогам заседания в Сеуле. ()\nСборную России следует исключить 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/gldsn/.local/share/virtualenvs/nti-ai-pipeline-LA9ij60T/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-2fad72a50e56>", line 6, in <module>
    trainer.fit(train_loader, val_loader, manager, max_step=50001, steps_betw_evals=5000, stop_patience=3)
  File "./../../../nti_ai_pipeline/pipeline/training/train.py", line 40, in fit
    loss_val = self.weights_updater.fit_with_batch(model_manager, batch)
  File "./../../../nti_ai_pipeline/pipeline/training/weights_updater.py", line 56, in fit_with_batch
    loss = self._calc_loss(manager, inputs, labels)
  File "./../../../nti_ai_pipeline/pipeline/training/weights_updater.py", line 67, in _calc_loss
    preds, labels = manager.preproc_forward(inputs, labels)
  File "./../../../nti_ai_pipeline/pipeline/modeling/managing_model.py", line 46, in preproc_forward
    preds_raw = self.mo

TypeError: object of type 'NoneType' has no len()

In [ ]:
# best_manager = trainer.load_best_manager()
# best_manager = managing_model.ModelManager.load(saver, '9ab8822d-1d50-4d8d-904b-2e7a70dca78f')
submitter = rucos_modeling.RucosSubmitter(subm_dir=os.path.join(BASE_DIR, 'submissions'))

test_container = nti_rucos.data.RucosContainer(path=os.path.join(DATA_DIR, 'rucos_test.jsonl'), has_labels=False,
                                                  query_placeholder_union_mode="concatenate",
                                                  )
test_dataset = nti_rucos.data.RucosSubmDataset(test_container, switch_texts=True)
# test_dataset = torch_data.Subset(test_dataset, np.arange(1000))
test_loader = torch_data.DataLoader(test_dataset)

submitter.create_submission(
    best_manager, test_loader, subm_file_name="subm_single_bert_50000_steps.jsonl",
    )